Below is an example usage of the animator module:

Note: animator.py must be uploaded to the same directory as the notebook for the below code to function properly

In [ ]:
import animator
import numpy as np

# Set the names of the curves and the x- and y-limits for the animation
a = animator.Animation(['Re(Ψ)', 'Im(Ψ)', '|Ψ|^2'],
                       (-2*np.pi, 2*np.pi), (-10, 10))

# define an "animate" function which takes in the integer frame number,
# calls Animation.set_data to data and returns Animation.lines
x = np.arange(-2*np.pi, 2*np.pi, 0.01)
def animate1(i):
  a.set_data(0, x, np.sin(x * 1.5**(i / 100)))
  a.set_data(1, x, np.sin(x * 1.5**(i / 50)))
  a.set_data(2, x, np.sin(x * 1.5**(i / 25)))
  return a.lines

# set the animate function and make the animation
a.set_anim_func(animate1)
ani1 = a.make_anim(frames=100, interval=20, blit=True)

# a.save('ani1.mp4')     # uncomment this line to save to a mp4 file

# if the last line in a notebook cell is a matplotlib FuncAnimation object,
# which is what Animation.make_anim returns, settings in the Animation object's
# constructor cause it to be output directly, so simply type the variable name
# to output your animation
ani1

In [ ]:
# after an object has been created, it's animate function can be modified
def animate2(i):
  a.set_data(0, x, np.sin(x * 1.5**(-i / 100)))
  a.set_data(1, x, np.sin(x * 1.5**(-i / 50)))
  a.set_data(2, x, np.sin(x * 1.5**(-i / 25)))
  return a.lines

# the output of Animation.make_anim does not need to be stored into
# a variable to display the animation, so long as it is the last
# entry in the cell, as seen below
a.set_anim_func(animate2)
a.make_anim(frames=100, interval=20, blit=True)

Note that only one animation can be embed at a time based on how they're displayed in a notebook.

Below are the contents of animator.py for better understanding of features:
```
class Animation:
  # allow for html embedded animations, and import matplotlib
  from matplotlib import rc
  rc('animation', html='jshtml')

  import matplotlib.pyplot as plt
  import matplotlib.animation as anim

  def __init__(self, names: list[str], xlim: tuple, ylim: tuple):
    if not isinstance(names, list):
      raise TypeError("names must be a list of strings")
    for name in names:
      if not isinstance(name, str):
        raise TypeError("names must be a list of strings")
    if not isinstance(xlim, tuple) or len(xlim) != 2:
      raise TypeError("xlim must be a 2-tuple of floats")
    if not isinstance(ylim, tuple) or len(ylim) != 2:
      raise TypeError("ylim must be a 2-tuple of floats")
    
    # set up plot
    self.fig, self.ax = Animation.plt.subplots()

    self.lines = []
    for name in names:
      line, = self.ax.plot([], [], lw = 3)
      self.lines.append(line)
    
    self.ax.legend([ line for line in self.lines ], [ name for name in names ])
    self.ax.set_xlim(xlim[0], xlim[1])
    self.ax.set_ylim(ylim[0], ylim[1])

    # for some reason subplots() is showing a plot, so hide as not required
    self.plt.close()
  
  def set_data(self, lineNum: int, x: list[float], y: list[float]):
    self.lines[lineNum].set_data(x, y)
  
  # func: (frame: int) -> a.lines
  def set_anim_func(self, func):
    self.func = func
  
  # frames = number of frames
  # interval = time between frames in ms
  def make_anim(self, frames: int, interval: int, blit: bool):
    self.anim = Animation.anim.FuncAnimation(self.fig,
                                             self.func,
                                             frames=frames,
                                             interval=interval,
                                             blit=blit)
    return self.anim

  # unable to have a "show()" function with html embed, so just save the output
  # of make_anim to a variable and type the name of that variable to show
  #
  # ex:
  #   >>> ani = a.make_anim(frames=100, interval=20, blit=True)
  #   >>> ani

  def save(self, path: str):
    if not isinstance(path, str):
      raise TypeError("path must be a string")
    self.anim.save(path)
```